In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("Hyperparameter optimization") \
  .getOrCreate()

24/10/13 22:49:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
data = spark.read.csv("data/hotel-reservations.csv", header=True, inferSchema=True)

In [3]:
stages = []

In [4]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder 

categorical_cols = ['type_of_meal_plan', 'room_type_reserved', 'arrival_month', 'market_segment_type']

for col in categorical_cols:
    string_indexer = StringIndexer(inputCol=col, outputCol= col + '_index')
    print(f'StringIndexer {string_indexer.getInputCol()} -> {string_indexer.getOutputCol()}')
    
    encoder = OneHotEncoder(inputCol=string_indexer.getOutputCol(), outputCol=col + '_vec')
    print(f'OneHotEncoder {encoder.getInputCol()} -> {encoder.getOutputCol()}')
    print()
    
    stages += [string_indexer, encoder]

StringIndexer type_of_meal_plan -> type_of_meal_plan_index
OneHotEncoder type_of_meal_plan_index -> type_of_meal_plan_vec

StringIndexer room_type_reserved -> room_type_reserved_index
OneHotEncoder room_type_reserved_index -> room_type_reserved_vec

StringIndexer arrival_month -> arrival_month_index
OneHotEncoder arrival_month_index -> arrival_month_vec

StringIndexer market_segment_type -> market_segment_type_index
OneHotEncoder market_segment_type_index -> market_segment_type_vec



In [5]:
stages += [StringIndexer(inputCol='booking_status', outputCol= 'booking_status_index')]

In [6]:
encoded_categorical_cols = [col + "_vec" for col in categorical_cols]
encoded_categorical_cols

['type_of_meal_plan_vec',
 'room_type_reserved_vec',
 'arrival_month_vec',
 'market_segment_type_vec']

In [7]:
numeric_cols = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights',
               'required_car_parking_space', 'lead_time', 'arrival_year', 'arrival_date',
               'repeated_guest', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
               'avg_price_per_room', 'no_of_special_requests'] 

In [8]:
input_columns = encoded_categorical_cols + numeric_cols
input_columns

['type_of_meal_plan_vec',
 'room_type_reserved_vec',
 'arrival_month_vec',
 'market_segment_type_vec',
 'no_of_adults',
 'no_of_children',
 'no_of_weekend_nights',
 'no_of_week_nights',
 'required_car_parking_space',
 'lead_time',
 'arrival_year',
 'arrival_date',
 'repeated_guest',
 'no_of_previous_cancellations',
 'no_of_previous_bookings_not_canceled',
 'avg_price_per_room',
 'no_of_special_requests']

In [9]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=input_columns, outputCol='features')

stages += [assembler]

In [10]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
print("Train size: ", train_data.count())
print("Test size: ", test_data.count())

Train size:  29040
Test size:  7235


In [11]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(featuresCol='features', labelCol='booking_status_index', maxIter=10)

stages.append(gbt)

In [12]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=stages)

In [13]:
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [5, 10]) \
    .addGrid(gbt.maxIter, [10, 20]) \
    .addGrid(gbt.stepSize, [0.1, 0.2]) \
    .build()

evaluator = MulticlassClassificationEvaluator(
    labelCol='booking_status_index',
    predictionCol='prediction',
    metricName='accuracy'
)

tvs_model = TrainValidationSplit(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    seed=42
)

best_model = tvs_model.fit(train_data)

24/10/13 22:50:51 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/10/13 22:50:51 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/10/13 22:51:06 WARN DAGScheduler: Broadcasting large task binary with size 1009.6 KiB
24/10/13 22:51:06 WARN DAGScheduler: Broadcasting large task binary with size 1031.6 KiB
24/10/13 22:51:06 WARN DAGScheduler: Broadcasting large task binary with size 1066.4 KiB
24/10/13 22:51:06 WARN DAGScheduler: Broadcasting large task binary with size 1061.6 KiB
24/10/13 22:51:15 WARN DAGScheduler: Broadcasting large task binary with size 1009.6 KiB
24/10/13 22:51:16 WARN DAGScheduler: Broadcasting large task binary with size 1031.6 KiB
24/10/13 22:51:16 WARN DAGScheduler: Broadcasting large task binary with size 1066.4 KiB
24/10/13 22:51:16 WARN DAGScheduler: Broadcasting large task binary with size 1064.1 KiB
24/10/13 22:51:16 WARN DAGScheduler: Broadcasting large task binary wit

In [14]:
predictions = best_model.transform(test_data)

In [15]:
accuracy = evaluator.evaluate(predictions)
print(f'Test Accuracy = {accuracy}')

Test Accuracy = 0.8800276434001382


24/10/13 22:51:58 WARN DAGScheduler: Broadcasting large task binary with size 1971.0 KiB


In [16]:
for param_values, metric_value in zip(paramGrid, best_model.avgMetrics):
    for param, value in param_values.items():
        print(f'{param.name} = {value}')
    print(f'Accuracy -> {metric_value}')


AttributeError: 'TrainValidationSplitModel' object has no attribute 'avgMetrics'